In [ ]:
%pip install -U pandas numpy scikit-learn matplotlib openpyxl scikeras

In [ ]:
import sys, tensorflow as tf
print(sys.executable)
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))

In [ ]:
import tensorflow as tf
print("TF:", tf.__version__)
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))

tf.debugging.set_log_device_placement(True)  


a = tf.random.normal([4000, 4000])
b = tf.random.normal([4000, 4000])
c = tf.matmul(a, b)
_ = c.numpy() 

In [ ]:

import os, random, numpy as np, pandas as pd, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers


random.seed(42); np.random.seed(42); tf.random.set_seed(42)


os.makedirs("outputs", exist_ok=True)

print("TensorFlow:", tf.__version__)

In [ ]:

TARGET = "Grid  Power"  
FILTER_Y_EQ_ZERO = True  


raw_data = pd.read_excel('raw data.xlsx')


raw_data = raw_data.drop(columns=['times'], errors='ignore').copy()


if FILTER_Y_EQ_ZERO:
    raw_data = raw_data[raw_data[TARGET] != 0].copy()


num_cols = [c for c in raw_data.columns if c != TARGET]
raw_data[num_cols] = raw_data[num_cols].apply(pd.to_numeric, errors='coerce')
raw_data = raw_data.dropna().reset_index(drop=True)

print("Data shape:", raw_data.shape)
display(raw_data.head(3))


In [ ]:


train_dataset = raw_data.sample(frac=0.8, random_state=0)
test_dataset  = raw_data.drop(train_dataset.index)


test_x_data = test_dataset.drop(columns=[TARGET]).copy()
test_y_data = test_dataset[TARGET].copy()


labeled_train_data   = train_dataset.sample(frac=0.6, random_state=0).copy()
unlabeled_train_data = train_dataset.drop(labeled_train_data.index).copy()


unlabeled_train_data_actual = unlabeled_train_data.pop(TARGET).copy()
labeled_data_labels         = labeled_train_data.pop(TARGET).copy()

len(train_dataset), len(labeled_train_data), len(unlabeled_train_data), len(test_dataset)


In [ ]:

from sklearn.preprocessing import StandardScaler


print("Columns:", train_dataset.columns.tolist())


feature_cols = [c for c in train_dataset.columns if c != TARGET]


scaler_fs = StandardScaler().fit(labeled_train_data[feature_cols])


X_fs_tr = scaler_fs.transform(labeled_train_data[feature_cols]).astype("float32")
y_fs_tr = labeled_data_labels.values.astype("float32")  


X_fs_te = scaler_fs.transform(test_dataset[feature_cols]).astype("float32")
y_fs_te = test_dataset[TARGET].values.astype("float32")

INPUT_DIM_FS = X_fs_tr.shape[1]
print("Supervised shapes → X_tr:", X_fs_tr.shape, " X_te:", X_fs_te.shape, "  INPUT_DIM_FS:", INPUT_DIM_FS)


In [ ]:

def build_model(
    input_dim: int,
    layers_: int = 3,
    units: int = 32,
    dropout: float = 0.0,
    l2: float = 0.0,
    activation: str = "relu",      
    learning_rate: float = 1e-3,
    loss: str = "mae",
    metrics=("mae","mse"),
):
    inputs = keras.Input(shape=(input_dim,))
    x = inputs
    kreg = regularizers.l2(l2) if (l2 and l2 > 0) else None
    for _ in range(layers_):
        x = layers.Dense(units, kernel_regularizer=kreg)(x)
        if activation == "leaky_relu":
            x = layers.LeakyReLU(alpha=0.2)(x)
        else:
            x = layers.Activation(activation)(x)
        if dropout and dropout > 0:
            x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=list(metrics),
    )
    return model


In [ ]:


callbacks_fs = [
    keras.callbacks.EarlyStopping(monitor="val_mae", patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5, min_lr=1e-5),
]

model_fs = build_model(
    INPUT_DIM_FS,
    layers_=3, units=32, dropout=0.0, l2=0.0,
    activation="relu", learning_rate=1e-3, loss="mae"
)

history_fs = model_fs.fit(
    X_fs_tr, y_fs_tr,
    validation_split=0.2,
    epochs=100, batch_size=64, verbose=0,
    callbacks=callbacks_fs
)


In [ ]:


from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error


val_fraction = 0.2
n_all = len(X_fs_tr)
n_val = int(round(n_all * val_fraction))
idx_tr_in = slice(0, n_all - n_val)
idx_va_in = slice(n_all - n_val, n_all)

X_tr_in, y_tr_in = X_fs_tr[idx_tr_in], y_fs_tr[idx_tr_in]
X_va_in, y_va_in = X_fs_tr[idx_va_in], y_fs_tr[idx_va_in]


yhat_tr = model_fs.predict(X_tr_in, verbose=0).ravel()
yhat_va = model_fs.predict(X_va_in, verbose=0).ravel()
yhat_te = model_fs.predict(X_fs_te, verbose=0).ravel()

def metrics_block(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)

    loss = mae
    return {"Loss": loss, "MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2, "MAPE": mape}

m_tr = metrics_block(y_tr_in, yhat_tr)
m_va = metrics_block(y_va_in, yhat_va)
m_te = metrics_block(y_fs_te, yhat_te)


hist_df = pd.DataFrame(history_fs.history).copy()
hist_df.insert(0, "epoch", np.arange(1, len(hist_df) + 1))


pred_train_df = pd.DataFrame({"y_true": y_tr_in, "y_pred": yhat_tr, "residual": y_tr_in - yhat_tr})
pred_val_df   = pd.DataFrame({"y_true": y_va_in, "y_pred": yhat_va, "residual": y_va_in - yhat_va})
pred_test_df  = pd.DataFrame({"y_true": y_fs_te, "y_pred": yhat_te, "residual": y_fs_te - yhat_te})


summary_df = pd.DataFrame.from_dict({
    "Train": m_tr,
    "Val":   m_va,
    "Test":  m_te,
}, orient="index")[["Loss","MAE","MSE","RMSE","R2","MAPE"]]


os.makedirs("outputs", exist_ok=True)


pd.DataFrame({"y_true": y_fs_te, "y_pred": yhat_te}).to_excel(
    "outputs/fully_supervised_test_preds.xlsx", index=False
)


with pd.ExcelWriter("outputs/fully_supervised_errors.xlsx", engine="openpyxl") as writer:
    hist_df.to_excel(writer, sheet_name="history", index=False)    
    summary_df.to_excel(writer, sheet_name="summary")            
    pred_train_df.to_excel(writer, sheet_name="pred_train", index=False)
    pred_val_df.to_excel(writer,   sheet_name="pred_val",   index=False)
    pred_test_df.to_excel(writer,  sheet_name="pred_test",  index=False)

print("[Fully-Supervised DNN] Summary:\n", summary_df)
